In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from matplotlib import pyplot as plt

import geopandas as gpd
from shapely import wkt

In [3]:
import cartoframes
from cartoframes.auth import set_default_credentials
from cartoframes.viz import Map, Layer, popup_element, size_bins_style

from cartoframes.viz import Map, Layer, basemaps, popup_element, basic_style, color_category_style, color_bins_style, default_legend, color_bins_legend, category_widget, formula_widget,  histogram_widget

set_default_credentials('https://cartoframes.carto.com')

In [4]:
df = pd.read_csv("Final_merged_city_portland_with_NAICS.csv")

df = df.drop(columns = ["Unnamed: 0"])

df = df.drop_duplicates()

df['geometry'] = df['geometry'].apply(wkt.loads)
df['centroid'] = df['centroid'].apply(wkt.loads)
df = gpd.GeoDataFrame(df, geometry='geometry')

In [5]:
replace_values = {11:'Agriculture, Forestry, Fishing and Hunting',
                  12:'Mining, Quarrying, and Oil and Gas Extraction',
                  22:'Utilities',
                  23:'Construction',
                  31:'Manufacturing',
                  32:'Manufacturing',
                  33:'Manufacturing',
                  42:'Wholesale Trade',
                  44:'Retail Trade',
                  45:'Retail Trade',
                  48:'Transportation and Warehousing',
                  49:'Transportation and Warehousing',
                  51:'Information',
                  52:'Finance and Insurance',
                  53:'Real Estate and Rental and Leasing',
                  54:'Professional, Scientific, and Technical Services',
                  55:'Management of Companies and Enterprises',
                  56:'Administrative and Support and Waste Management and Remediation Services',
                  61:'Educational Services',
                  62:'Health Care and Social Assistance',
                  71:'Arts, Entertainment, and Recreation',
                  72:'Accommodation and Food Services',
                  81:'Other Services (except Public Administration)'}

In [6]:
df['maxNaics_est_cat'] = df['maxNaics_est']

df = df.replace({"maxNaics_est_cat": replace_values})

df['maxNaics_est_cat'] = pd.Categorical(df.maxNaics_est_cat)

In [7]:
df.head()

,Unnamed: 0.1,geo_id,centroid,year,geometry,total_tran,pop,edut,collegemale,collegefemale,households,income,zipcode,Empratio,estabratio,ratio_65,ratio_22_50,bus_ratio,bus_ratio_total,public_ratio,car_ratio,bicy_ratio,walked_ratio,maxNaics_est,totalEstab,maxNaics_emp,maxNaics_est_cat
0,2432,41051010402,POINT (-122.3356996936241 45.48566422279789),2010,"MULTIPOLYGON (((-122.38688 45.51903, -122.3726...",2601.0,5717.0,3699.0,294.0,356.0,162.0,1859.0,97009,0.0,0.0,0.391464,0.091307,0.490196,0.009612,0.019608,0.891580,0.005767,0.014994,23,95,23,Construction
1,2435,41051010402,POINT (-122.3356996936241 45.48566422279789),2011,"MULTIPOLYGON (((-122.38688 45.51903, -122.3726...",2826.0,6037.0,4024.0,368.0,476.0,149.0,1960.0,97009,0.0,0.0,0.405831,0.108001,0.305970,0.014508,0.047417,0.866596,0.030432,0.009908,23,92,23,Construction
2,2438,41051010402,POINT (-122.3356996936241 45.48566422279789),2012,"MULTIPOLYGON (((-122.38688 45.51903, -122.3726...",2868.0,6153.0,4090.0,369.0,507.0,139.0,1998.0,97009,0.0,0.0,0.393954,0.109052,0.266667,0.011158,0.041841,0.857043,0.033473,0.004533,23,85,23,Construction
3,2441,41051010402,POINT (-122.3356996936241 45.48566422279789),2013,"MULTIPOLYGON (((-122.38688 45.51903, -122.3726...",2876.0,6022.0,3931.0,296.0,404.0,129.0,2087.0,97009,0.0,0.0,0.366490,0.108602,0.321739,0.012865,0.039986,0.858832,0.025730,0.012517,23,86,23,Construction
4,2444,41051010402,POINT (-122.335683996136 45.48565935229686),2014,"MULTIPOLYGON (((-122.38688 45.51903, -122.3726...",3078.0,6181.0,4094.0,316.0,457.0,120.0,2191.0,97009,0.0,0.0,0.387316,0.108720,0.264368,0.007472,0.028265,0.875569,0.016244,0.023392,23,89,23,Construction


In [8]:
df2 = df[['geo_id','year','total_tran','pop','edut', 'collegemale', 'collegefemale', 'households', 'income',
       'zipcode', 'Empratio', 'estabratio', 'ratio_65', 'ratio_22_50',
       'bus_ratio', 'bus_ratio_total', 'public_ratio', 'car_ratio',
       'bicy_ratio', 'walked_ratio', 'maxNaics_est', 'totalEstab',
       'maxNaics_emp']]

In [9]:
df2

,geo_id,year,total_tran,pop,edut,collegemale,collegefemale,households,income,zipcode,Empratio,estabratio,ratio_65,ratio_22_50,bus_ratio,bus_ratio_total,public_ratio,car_ratio,bicy_ratio,walked_ratio,maxNaics_est,totalEstab,maxNaics_emp
0,41051010402,2010,2601.0,5717.0,3699.0,294.0,356.0,162.0,1859.0,97009,0.000000,0.000000,0.391464,0.091307,0.490196,0.009612,0.019608,0.891580,0.005767,0.014994,23,95,23
1,41051010402,2011,2826.0,6037.0,4024.0,368.0,476.0,149.0,1960.0,97009,0.000000,0.000000,0.405831,0.108001,0.305970,0.014508,0.047417,0.866596,0.030432,0.009908,23,92,23
2,41051010402,2012,2868.0,6153.0,4090.0,369.0,507.0,139.0,1998.0,97009,0.000000,0.000000,0.393954,0.109052,0.266667,0.011158,0.041841,0.857043,0.033473,0.004533,23,85,23
3,41051010402,2013,2876.0,6022.0,3931.0,296.0,404.0,129.0,2087.0,97009,0.000000,0.000000,0.366490,0.108602,0.321739,0.012865,0.039986,0.858832,0.025730,0.012517,23,86,23
4,41051010402,2014,3078.0,6181.0,4094.0,316.0,457.0,120.0,2191.0,97009,0.000000,0.000000,0.387316,0.108720,0.264368,0.007472,0.028265,0.875569,0.016244,0.023392,23,89,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,41051008500,2017,2214.0,4518.0,3076.0,96.0,223.0,124.0,1477.0,97266,758.805395,55.321773,0.414564,0.137450,0.954733,0.104788,0.109756,0.837398,0.000000,0.015357,72,122,72
4996,41051008600,2017,2217.0,4008.0,2901.0,152.0,278.0,125.0,1612.0,97266,289.441233,21.102119,0.449850,0.093313,1.000000,0.114118,0.114118,0.746504,0.009923,0.009021,72,122,72
4997,41051008901,2017,4074.0,9634.0,6682.0,610.0,560.0,161.0,3108.0,97266,39.113680,2.851638,0.365165,0.148952,0.567227,0.033137,0.058419,0.873098,0.003927,0.021846,72,122,72
4998,41051008902,2017,2146.0,3974.0,2720.0,352.0,297.0,119.0,1450.0,97266,797.919075,58.173410,0.483392,0.104177,0.745665,0.060112,0.080615,0.769804,0.032619,0.046598,72,122,72


In [12]:
sorted_df2 = df2.sort_values(['geo_id', 'year'], ascending=[True, True])

In [14]:
sorted_df2.tail(20)

,geo_id,year,total_tran,pop,edut,collegemale,collegefemale,households,income,zipcode,Empratio,estabratio,ratio_65,ratio_22_50,bus_ratio,bus_ratio_total,public_ratio,car_ratio,bicy_ratio,walked_ratio,maxNaics_est,totalEstab,maxNaics_emp
2015,41051010600,2017,1012.0,3007.0,2591.0,263.0,277.0,174.0,1964.0,97201,97.624085,4.858421,0.418357,0.186897,0.656977,0.22332,0.339921,0.188735,0.034585,0.365613,54,241,54
2527,41051010600,2017,1012.0,3007.0,2591.0,263.0,277.0,174.0,1964.0,97204,29180.466667,1193.040370,0.418357,0.186897,0.656977,0.22332,0.339921,0.188735,0.034585,0.365613,54,539,54
2551,41051010600,2017,1012.0,3007.0,2591.0,263.0,277.0,174.0,1964.0,97205,17404.318075,819.535158,0.418357,0.186897,0.656977,0.22332,0.339921,0.188735,0.034585,0.365613,54,331,54
2855,41051010600,2017,1012.0,3007.0,2591.0,263.0,277.0,174.0,1964.0,97209,271.153435,14.490210,0.418357,0.186897,0.656977,0.22332,0.339921,0.188735,0.034585,0.365613,54,324,54
3573,41051980000,2010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,97217,3247.120879,122.597802,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,72,144,42
4236,41051980000,2010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,97227,184.106729,5.968677,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,62,66,62
3587,41051980000,2011,5.0,6.0,1.0,0.0,0.0,0.0,0.0,97217,3199.648352,117.257143,0.500000,0.000000,0.000000,0.00000,0.000000,0.600000,0.000000,0.400000,44,142,48
4241,41051980000,2011,5.0,6.0,1.0,0.0,0.0,0.0,0.0,97227,178.016241,6.368910,0.500000,0.000000,0.000000,0.00000,0.000000,0.600000,0.000000,0.400000,62,67,62
3601,41051980000,2012,4.0,5.0,1.0,0.0,0.0,0.0,0.0,97217,1925.010989,86.993407,0.600000,0.000000,0.000000,0.00000,0.000000,0.500000,0.000000,0.500000,72,141,55
4246,41051980000,2012,4.0,5.0,1.0,0.0,0.0,0.0,0.0,97227,162.354988,5.690255,0.600000,0.000000,0.000000,0.00000,0.000000,0.500000,0.000000,0.500000,62,76,62


In [15]:
sorted_df = df.sort_values(['geo_id', 'year'], ascending=[True, True])

In [16]:
sorted_df

,Unnamed: 0.1,geo_id,centroid,year,geometry,total_tran,pop,edut,collegemale,collegefemale,households,income,zipcode,Empratio,estabratio,ratio_65,ratio_22_50,bus_ratio,bus_ratio_total,public_ratio,car_ratio,bicy_ratio,walked_ratio,maxNaics_est,totalEstab,maxNaics_emp,maxNaics_est_cat
2016,0,41051000100,POINT (-122.6585878427874 45.47691494424082),2010,"MULTIPOLYGON (((-122.66851 45.48963, -122.6654...",3362.0,6043.0,4558.0,647.0,624.0,167.0,2860.0,97202,4556.326728,256.721605,0.499421,0.105080,0.965699,0.108864,0.112731,0.722784,0.077930,0.047591,72,173,31,Accommodation and Food Services
2017,0,41051000100,POINT (-122.6585878427874 45.47691494424082),2010,"MULTIPOLYGON (((-122.66851 45.48963, -122.6654...",3362.0,6043.0,4558.0,647.0,624.0,167.0,2860.0,97202,4556.326728,256.721605,0.499421,0.105080,0.965699,0.108864,0.112731,0.722784,0.077930,0.047591,72,173,72,Accommodation and Food Services
2046,1,41051000100,POINT (-122.6585878427874 45.47691494424082),2011,"MULTIPOLYGON (((-122.66851 45.48963, -122.6654...",3260.0,6077.0,4625.0,597.0,687.0,171.0,2886.0,97202,4650.536491,262.502658,0.502386,0.101695,0.965608,0.111963,0.115951,0.708896,0.074233,0.041411,72,176,72,Accommodation and Food Services
2076,2,41051000100,POINT (-122.6585878427874 45.47691494424082),2012,"MULTIPOLYGON (((-122.66851 45.48963, -122.6654...",3096.0,5903.0,4478.0,551.0,698.0,169.0,2822.0,97202,3697.733204,211.971967,0.477554,0.102490,1.000000,0.092377,0.092377,0.715762,0.084625,0.035530,72,178,72,Accommodation and Food Services
2106,3,41051000100,POINT (-122.6585878427874 45.47691494424082),2013,"MULTIPOLYGON (((-122.66851 45.48963, -122.6654...",3111.0,6104.0,4596.0,564.0,763.0,168.0,2779.0,97202,3879.729338,211.757854,0.475754,0.112713,1.000000,0.093218,0.093218,0.646095,0.109932,0.056252,72,178,72,Accommodation and Food Services
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4261,2603,41051980000,POINT (-122.7060032163737 45.56039797898389),2015,"MULTIPOLYGON (((-122.72327 45.57101, -122.7220...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,97227,177.320186,6.595128,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,62,85,62,Health Care and Social Assistance
3657,2604,41051980000,POINT (-122.7060032163737 45.56039797898389),2016,"MULTIPOLYGON (((-122.72327 45.57101, -122.7220...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,97217,2305.978022,100.167033,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,72,163,42,Accommodation and Food Services
4266,2605,41051980000,POINT (-122.7060032163737 45.56039797898389),2016,"MULTIPOLYGON (((-122.72327 45.57101, -122.7220...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,97227,165.661253,6.769142,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,62,84,62,Health Care and Social Assistance
3671,2606,41051980000,POINT (-122.7060554883347 45.5604280058481),2017,"MULTIPOLYGON (((-122.72323 45.57125, -122.7241...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,97217,1529.802198,100.167033,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,72,162,72,Accommodation and Food Services


In [19]:
sorted_df2 = sorted_df2.drop_duplicates()

In [20]:
sorted_df2.shape

(2714, 23)

In [22]:
copy_df = sorted_df

In [28]:
sorted_df.loc[sorted_df['maxNaics_est'] == 72].head(20)

,Unnamed: 0.1,geo_id,centroid,year,geometry,total_tran,pop,edut,collegemale,collegefemale,households,income,zipcode,Empratio,estabratio,ratio_65,ratio_22_50,bus_ratio,bus_ratio_total,public_ratio,car_ratio,bicy_ratio,walked_ratio,maxNaics_est,totalEstab,maxNaics_emp,maxNaics_est_cat
2016,0,41051000100,POINT (-122.6585878427874 45.47691494424082),2010,"MULTIPOLYGON (((-122.66851 45.48963, -122.6654...",3362.0,6043.0,4558.0,647.0,624.0,167.0,2860.0,97202,4556.326728,256.721605,0.499421,0.105080,0.965699,0.108864,0.112731,0.722784,0.077930,0.047591,72,173,31,Accommodation and Food Services
2017,0,41051000100,POINT (-122.6585878427874 45.47691494424082),2010,"MULTIPOLYGON (((-122.66851 45.48963, -122.6654...",3362.0,6043.0,4558.0,647.0,624.0,167.0,2860.0,97202,4556.326728,256.721605,0.499421,0.105080,0.965699,0.108864,0.112731,0.722784,0.077930,0.047591,72,173,72,Accommodation and Food Services
2046,1,41051000100,POINT (-122.6585878427874 45.47691494424082),2011,"MULTIPOLYGON (((-122.66851 45.48963, -122.6654...",3260.0,6077.0,4625.0,597.0,687.0,171.0,2886.0,97202,4650.536491,262.502658,0.502386,0.101695,0.965608,0.111963,0.115951,0.708896,0.074233,0.041411,72,176,72,Accommodation and Food Services
2076,2,41051000100,POINT (-122.6585878427874 45.47691494424082),2012,"MULTIPOLYGON (((-122.66851 45.48963, -122.6654...",3096.0,5903.0,4478.0,551.0,698.0,169.0,2822.0,97202,3697.733204,211.971967,0.477554,0.102490,1.000000,0.092377,0.092377,0.715762,0.084625,0.035530,72,178,72,Accommodation and Food Services
2106,3,41051000100,POINT (-122.6585878427874 45.47691494424082),2013,"MULTIPOLYGON (((-122.66851 45.48963, -122.6654...",3111.0,6104.0,4596.0,564.0,763.0,168.0,2779.0,97202,3879.729338,211.757854,0.475754,0.112713,1.000000,0.093218,0.093218,0.646095,0.109932,0.056252,72,178,72,Accommodation and Food Services
2136,4,41051000100,POINT (-122.6585712638274 45.47692206766484),2014,"MULTIPOLYGON (((-122.66851 45.48963, -122.6654...",3148.0,5930.0,4561.0,660.0,735.0,172.0,2777.0,97202,4021.043983,215.397777,0.456998,0.119224,1.000000,0.073698,0.073698,0.663596,0.118806,0.024460,72,173,72,Accommodation and Food Services
2166,5,41051000100,POINT (-122.6585712638274 45.47692206766484),2015,"MULTIPOLYGON (((-122.66851 45.48963, -122.6654...",3155.0,6037.0,4723.0,547.0,765.0,171.0,2772.0,97202,4226.592557,225.246979,0.435812,0.132516,1.000000,0.074168,0.074168,0.637401,0.116640,0.031062,72,186,72,Accommodation and Food Services
2196,6,41051000100,POINT (-122.6585670655088 45.47691587188954),2016,"MULTIPOLYGON (((-122.66851 45.48963, -122.6654...",3323.0,6293.0,4873.0,629.0,756.0,171.0,2840.0,97202,4449.270179,244.303045,0.431114,0.147783,0.865900,0.068011,0.078543,0.645802,0.104725,0.041529,72,196,72,Accommodation and Food Services
2226,7,41051000100,POINT (-122.6585670655088 45.47691587188954),2017,"MULTIPOLYGON (((-122.66851 45.48963, -122.6654...",3491.0,6380.0,4924.0,725.0,837.0,172.0,2874.0,97202,3269.507008,241.305462,0.439498,0.148119,0.827273,0.052134,0.063019,0.683472,0.077055,0.056431,72,209,72,Accommodation and Food Services
2018,8,41051000200,POINT (-122.6442512670287 45.47419750748161),2010,"MULTIPOLYGON (((-122.65115 45.49032, -122.6493...",3223.0,6052.0,4245.0,582.0,664.0,179.0,2845.0,97202,1820.473659,102.572740,0.499670,0.125744,1.000000,0.144276,0.144276,0.696556,0.053987,0.046540,72,173,31,Accommodation and Food Services


In [25]:
temp = sorted_df.loc[sorted_df['zipcode'] == 97217]
temp['maxNaics_est_cat'].unique()

[Accommodation and Food Services, Retail Trade]
Categories (2, object): [Accommodation and Food Services, Retail Trade]

In [21]:
Map(Layer(sorted_df,color_category_style('maxNaics_est_cat'),popup_hover=[popup_element('zipcode','ZipCode')],title='Industry'),basemap=basemaps.voyager)